In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
import os
from zipfile import ZipFile

# Define the path to your kaggle.json file
kaggle_json_path = r"C:\LinkedInJobPostings\kaggle(1).json"

In [4]:
import shutil

# Ensure the .kaggle directory exists
kaggle_dir = os.path.expanduser('~/.kaggle')
os.makedirs(kaggle_dir, exist_ok=True)

# Copy the kaggle.json file to the .kaggle directory
shutil.copy(kaggle_json_path, os.path.join(kaggle_dir, 'kaggle(1).json'))


print("kaggle(1).json has been copied to ~/.kaggle/ directory")

kaggle(1).json has been copied to ~/.kaggle/ directory


In [5]:
# Define the dataset and download path
dataset = 'arshkon/linkedin-job-postings' 
download_path =  r"C:\LinkedInJobPostings\dataset"

In [6]:
import kaggle 
# Ensure the download directory exists
os.makedirs(download_path, exist_ok=True)

# Download the dataset
kaggle.api.dataset_download_files(dataset, path=download_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/arshkon/linkedin-job-postings


In [7]:
# List files in the download directory to verify the actual file names
for root, dirs, files in os.walk(download_path):
    for file in files:
        print(os.path.join(root, file))

C:\LinkedInJobPostings\dataset\postings.csv
C:\LinkedInJobPostings\dataset\preprocessed_companies.csv
C:\LinkedInJobPostings\dataset\preprocessed_postings.csv
C:\LinkedInJobPostings\dataset\preprocessed_salaries.csv
C:\LinkedInJobPostings\dataset\TF-IDF.ipynb
C:\LinkedInJobPostings\dataset\companies\companies.csv
C:\LinkedInJobPostings\dataset\companies\company_industries.csv
C:\LinkedInJobPostings\dataset\companies\company_specialities.csv
C:\LinkedInJobPostings\dataset\companies\employee_counts.csv
C:\LinkedInJobPostings\dataset\jobs\benefits.csv
C:\LinkedInJobPostings\dataset\jobs\job_industries.csv
C:\LinkedInJobPostings\dataset\jobs\job_skills.csv
C:\LinkedInJobPostings\dataset\jobs\salaries.csv
C:\LinkedInJobPostings\dataset\mappings\industries.csv
C:\LinkedInJobPostings\dataset\mappings\skills.csv


In [8]:
# Paths to individual CSV files
datasets = {
    'postings': os.path.join(download_path, 'postings.csv'),
    'companies': os.path.join(download_path, 'companies', 'companies.csv'),
    'company_industries': os.path.join(download_path, 'companies', 'company_industries.csv'),
    'job_skills': os.path.join(download_path, 'jobs', 'job_skills.csv'),
    'skills': os.path.join(download_path, 'mappings', 'skills.csv')
}


In [9]:
# Load each CSV file into a pandas DataFrame
dataframes = {name: pd.read_csv(path) for name, path in datasets.items()}

# Function to print head, describe, info, and shape
def print_dataframe_info(df, name):
    print(f"Dataset: {name}")
    print("\nHead:")
    print(df.head(), "\n")
    print("Describe:")
    print(df.describe(), "\n")
    print("Info:")
    df.info()
    print("\nShape:")
    print(df.shape, "\n")
    print("="*80 + "\n")

    

# Print information for each DataFrame
for name, df in dataframes.items():
    print_dataframe_info(df, name)
    

Dataset: postings

Head:
     job_id            company_name  \
0    921716   Corcoran Sawyer Smith   
1   1829192                     NaN   
2  10998357  The National Exemplar    
3  23221523  Abrams Fensterman, LLP   
4  35982263                     NaN   

                                               title  \
0                              Marketing Coordinator   
1                  Mental Health Therapist/Counselor   
2                        Assitant Restaurant Manager   
3  Senior Elder Law / Trusts and Estates Associat...   
4                                 Service Technician   

                                         description  max_salary pay_period  \
0  Job descriptionA leading real estate firm in N...        20.0     HOURLY   
1  At Aspen Therapy and Wellness , we are committ...        50.0     HOURLY   
2  The National Exemplar is accepting application...     65000.0     YEARLY   
3  Senior Associate Attorney - Elder Law / Trusts...    175000.0     YEARLY   
4  Looki

# DATA PREPROCESSING


In [10]:
# Function to find and print missing values
def find_missing_values(dataframes):
    for name, df in dataframes.items():
        print(f"Missing values in {name}:")
        print(df.isnull().sum())
        print("="*80 + "\n")

# Find and print missing values
find_missing_values(dataframes)

Missing values in postings:
job_id                             0
company_name                    1719
title                              0
description                        7
max_salary                     94056
pay_period                     87776
location                           0
company_id                      1717
views                           1689
med_salary                    117569
min_salary                     94056
formatted_work_type                0
applies                       100529
original_listed_time               0
remote_allowed                108603
job_posting_url                    0
application_url                36665
application_type                   0
expiry                             0
closed_time                   122776
formatted_experience_level     29409
skills_desc                   121410
listed_time                        0
posting_domain                 39968
sponsored                          0
work_type                          0
currency  

In [11]:
#handling missing values
companies = dataframes['companies']
postings = dataframes['postings']
# Drop the irrelevant columns from the companies DataFrame
companies = companies.drop(columns=['zip_code', 'address'])
# Drop rows with any missing values in the companies DataFrame
companies = companies.dropna()
# Handle missing values in the postings dataset
postings = dataframes['postings'].copy()
postings = postings.dropna(subset=['company_name'])
postings['description'] = postings['description'].fillna('No description available')
postings['max_salary'] = postings['max_salary'].fillna(postings['max_salary'].median())
postings['med_salary'] = postings['med_salary'].fillna(postings['med_salary'].median())
postings['min_salary'] = postings['min_salary'].fillna(postings['min_salary'].median())
postings['pay_period'] = postings['pay_period'].fillna(postings['pay_period'].mode()[0])
postings = postings.dropna(subset=['company_id'])
postings['views'] = postings['views'].fillna(postings['views'].median())
postings['applies'] = postings['applies'].fillna(postings['applies'].median())
postings['remote_allowed'] = postings['remote_allowed'].fillna(postings['remote_allowed'].mode()[0])
postings['application_url'] = postings['application_url'].fillna('No application URL')
postings['formatted_experience_level'] = postings['formatted_experience_level'].fillna(postings['formatted_experience_level'].mode()[0])
postings['skills_desc'] = postings['skills_desc'].fillna('No skills listed')
postings['posting_domain'] = postings['posting_domain'].fillna('No domain')
postings['currency'] = postings['currency'].fillna(postings['currency'].mode()[0])
postings['compensation_type'] = postings['compensation_type'].fillna(postings['compensation_type'].mode()[0])

In [12]:
# Print the head of the cleaned companies DataFrame
print("Cleaned Companies DataFrame:")
print(companies.head())

# Print the head of the cleaned postings DataFrame
print("\nCleaned Postings DataFrame:")
print(postings.head())

Cleaned Companies DataFrame:
   company_id                        name  \
0        1009                         IBM   
1        1016               GE HealthCare   
2        1025  Hewlett Packard Enterprise   
3        1028                      Oracle   
4        1033                   Accenture   

                                         description  company_size  state  \
0  At IBM, we do more than work. We create. We cr...           7.0     NY   
1  Every day millions of people feel the impact o...           7.0      0   
2  Official LinkedIn of Hewlett Packard Enterpris...           7.0  Texas   
3  We’re a cloud technology company that provides...           7.0  Texas   
4  Accenture is a leading global professional ser...           7.0      0   

  country              city                                                url  
0      US  Armonk, New York               https://www.linkedin.com/company/ibm  
1      US           Chicago      https://www.linkedin.com/company/gehealthc

In [13]:
# Merge postings with companies on company_id
merged_df = postings.merge(companies, on='company_id', how='left')

# Merge the result with company_industries on company_id
merged_df = merged_df.merge(dataframes['company_industries'], on='company_id', how='left')

# Merge job_skills with skills on skill_abr
job_skills_combined_df = dataframes['job_skills'].merge(dataframes['skills'], on='skill_abr', how='left')

# Merge the main DataFrame with the combined job skills and skills DataFrame on job_id
final_df = merged_df.merge(job_skills_combined_df, on='job_id', how='left')

# Print the head of the final merged DataFrame
print("\nFinal Merged DataFrame Head:")
print(final_df.head())


Final Merged DataFrame Head:
     job_id            company_name  \
0    921716   Corcoran Sawyer Smith   
1    921716   Corcoran Sawyer Smith   
2  10998357  The National Exemplar    
3  10998357  The National Exemplar    
4  23221523  Abrams Fensterman, LLP   

                                               title  \
0                              Marketing Coordinator   
1                              Marketing Coordinator   
2                        Assitant Restaurant Manager   
3                        Assitant Restaurant Manager   
4  Senior Elder Law / Trusts and Estates Associat...   

                                       description_x  max_salary pay_period  \
0  Job descriptionA leading real estate firm in N...        20.0     HOURLY   
1  Job descriptionA leading real estate firm in N...        20.0     HOURLY   
2  The National Exemplar is accepting application...     65000.0     YEARLY   
3  The National Exemplar is accepting application...     65000.0     YEARLY   
4  

In [15]:
# Calculate the average salary
final_df['avg_salary'] = final_df[['max_salary','med_salary', 'min_salary']].mean(axis=1)

# Drop the max_salary, min_salary, and med_salary columns
final_df = final_df.drop(columns=['max_salary', 'min_salary', 'med_salary'])

# Print the head of the updated DataFrame to verify
print(final_df.head())

     job_id            company_name  \
0    921716   Corcoran Sawyer Smith   
1    921716   Corcoran Sawyer Smith   
2  10998357  The National Exemplar    
3  10998357  The National Exemplar    
4  23221523  Abrams Fensterman, LLP   

                                               title  \
0                              Marketing Coordinator   
1                              Marketing Coordinator   
2                        Assitant Restaurant Manager   
3                        Assitant Restaurant Manager   
4  Senior Elder Law / Trusts and Estates Associat...   

                                       description_x pay_period  \
0  Job descriptionA leading real estate firm in N...     HOURLY   
1  Job descriptionA leading real estate firm in N...     HOURLY   
2  The National Exemplar is accepting application...     YEARLY   
3  The National Exemplar is accepting application...     YEARLY   
4  Senior Associate Attorney - Elder Law / Trusts...     YEARLY   

            location  com

In [16]:
# Dropping unnecessary columns
final_df = final_df.drop(columns=['work_type', 'original_listed_time'])

In [17]:
# Check for missing values in the DataFrame
missing_values = final_df.isnull().sum()

# Print the columns with missing values
print("Missing values in each column:")
print(missing_values[missing_values > 0])

Missing values in each column:
closed_time      203536
name              10093
description_y     10093
company_size      10093
state             10093
country           10093
city              10093
url               10093
industry            267
skill_abr          1541
skill_name         1541
dtype: int64


In [18]:
# Drop the specified columns
final_df = final_df.drop(columns=['name', 'description_y', 'company_size', 'state', 'country', 'city', 'url', 'industry'])
final_df['closed_time'] = final_df['closed_time'].fillna(-1)
final_df['skill_abr'] = final_df['skill_abr'].fillna('Unknown Skill')
final_df['skill_name'] = final_df['skill_name'].fillna('Unknown Skill')
# Check for any remaining missing values
missing_values = final_df.isnull().sum()
print("Remaining missing values in the DataFrame:")
print(missing_values[missing_values > 0])

Remaining missing values in the DataFrame:
Series([], dtype: int64)


In [19]:
print(final_df.columns)

Index(['job_id', 'company_name', 'title', 'description_x', 'pay_period',
       'location', 'company_id', 'views', 'formatted_work_type', 'applies',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'currency', 'compensation_type',
       'skill_abr', 'skill_name', 'avg_salary'],
      dtype='object')


In [20]:
# Save the cleaned dataset to a CSV file
final_df.to_csv('cleaned_data.csv', index=False)

